In [10]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
%matplotlib inline

traindata = pd.read_csv("train_values.csv",sep=',')
traindata.columns[traindata.isna().any()]
data=traindata
data.drop(['building_id'], axis=1, inplace=True)
trainlabels = pd.read_csv("train_labels.csv",sep=',')
trainout=trainlabels
trainout.drop(['building_id'], axis=1, inplace=True)
testvalues = pd.read_csv("test_values.csv",sep=',')
testdata=testvalues
testdata.drop(['building_id'], axis=1, inplace=True)

In [11]:
cleanup_letters = {"land_surface_condition": {"n": 1, "o": 2, "t":3},
                "foundation_type": {"h": 1, "i": 2, "r": 3, "u": 4, "w": 5},               
                "roof_type": {"n": 1, "q": 2, "x": 3},
                "ground_floor_type": {"f": 1, "m": 2, "v": 3, "x": 4, "z": 5},
                "other_floor_type": {"j": 1, "q": 2, "s": 3, "x": 4},
                "position": {"j": 1, "o": 2, "s": 3, "t": 4},
                "plan_configuration": {"a": 1, "c": 2, "d": 3, "f": 4, "m": 5, "n": 6, "o": 7, "q": 8, "s": 9, "u":10},
                "legal_ownership_status": {"a": 1, "r": 2, "v": 3, "w": 4}             
               }
data.replace(cleanup_letters, inplace=True)
testdata.replace(cleanup_letters, inplace=True)
a = list(data)
del a[-1]
x = data[a].values
y = trainout['damage_grade'].values
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [12]:
clf = RandomForestClassifier(n_estimators=50,max_features=25,min_samples_leaf=5,n_jobs=-1,random_state=42)
clf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=25, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [13]:
from sklearn.metrics import f1_score
import numpy as np
y_pred0=clf.predict(x_test)
f1_model0=f1_score(y_test,y_pred0,average='micro',labels=np.unique(y_pred0))
print("F1 score Model 0 : ",f1_model0)

F1 score Model 0 :  0.7390495193875789


In [14]:
from sklearn.ensemble import GradientBoostingClassifier
model_1 = GradientBoostingClassifier()
model_1.fit(x_train,y_train)
y_pred1=model_1.predict(x_test)
f1_model1=f1_score(y_test,y_pred1,average='micro',labels=np.unique(y_pred1))
print("F1 score Model 1 : ",f1_model1)

F1 score Model 1 :  0.6825463824562077


In [15]:
from sklearn.neighbors import KNeighborsClassifier
model_2 = KNeighborsClassifier(n_neighbors=200)
model_2.fit(x_train,y_train)
y_pred2=model_2.predict(x_test)
f1_model2=f1_score(y_test,y_pred2,average='micro',labels=np.unique(y_pred2))
print("F1 score Model 2 : ",f1_model2)

F1 score Model 2 :  0.5927936916022333


In [18]:
from mlxtend.classifier import StackingClassifier
sclf = StackingClassifier(classifiers=[clf,model_1], meta_classifier=model_2)
sclf.fit(x_train,y_train)

StackingClassifier(average_probas=False,
          classifiers=[RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=25, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fract...     subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)],
          drop_last_proba=False,
          meta_classifier=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=200, p=2,
           weights='uniform'),
          store_train_meta_features=False, use_clones=True,
          use_features_in_secondary=False, use_probas=False, verbose=0)

In [19]:
y_predf=sclf.predict(x_test)
f1_modelf=f1_score(y_test,y_predf,average='micro',labels=np.unique(y_predf))
print("F1 score Model f : ",f1_modelf)

F1 score Model f :  0.7390495193875789
